In [ ]:
# --- مرحله 1: نصب وابستگی‌ها ---
!pip install pandas pyarrow scikit-learn joblib

# --- مرحله 2: متصل شدن به Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- مرحله 3: خواندن دیتاست با مدیریت خطا ---
import pandas as pd
import os

file_path = '/content/drive/MyDrive/formation_damage_optimized.parquet'

try:
    # سعی در خواندن فایل Parquet
    df = pd.read_parquet(file_path)
    print("✅ دیتاست با موفقیت خوانده شد.")
except FileNotFoundError:
    print("❌ فایل دیتاست یافت نشد. دیتای تست مصنوعی تولید می‌شود...")
    # تولید دیتای تست مصنوعی (در صورت نبود فایل)
    from generate_synthetic_data import generate_data
    df = generate_data(n_samples=1000)
    df.to_parquet('formation_damage_optimized.parquet')
    print("✅ دیتای تست مصنوعی ذخیره شد.")
except Exception as e:
    print(f"❌ خطای غیرمنتظره: {e}")
    exit()

# --- مرحله 4: شناسایی ستون‌های عددی ---
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
print("🔢 ستون‌های عددی شناسایی شده:", numeric_cols)

# --- مرحله 5: نرمال‌سازی و ذخیره پارامترها ---
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import joblib

scalers = {}
normalized_df = df.copy()

for col in numeric_cols:
    # تعیین روش نرمال‌سازی (مثال: Min-Max برای مقادیر محدود، Z-Score برای توزیع نرمال)
    if df[col].min() >= 0 and df[col].max() <= 1000:
        scaler = MinMaxScaler()
        normalized_df[col] = scaler.fit_transform(df[[col]])
        scalers[col] = {
            'type': 'MinMax',
            'min': scaler.data_min_[0],
            'max': scaler.data_max_[0]
        }
    else:
        scaler = StandardScaler()
        normalized_df[col] = scaler.fit_transform(df[[col]])
        scalers[col] = {
            'type': 'Standard',
            'mean': scaler.mean_[0],
            'std': scaler.scale_[0]
        }

# ذخیره اسکیلرهای آموزش دیده
joblib.dump(scalers, 'scalers_params.joblib')
print("✅ پارامترهای نرمال‌سازی ذخیره شدند.")

# --- مرحله 6: ذخیره داده نرمال‌سازی‌شده ---
normalized_df.to_parquet('normalized_formation_damage.parquet')
print("✅ داده نرمال‌سازی‌شده ذخیره شد.")



Mounted at /content/drive
✅ دیتاست با موفقیت خوانده شد.
🔢 ستون‌های عددی شناسایی شده: ['Temperature_C', 'Pressure_psi', 'pH', 'Salinity_ppm', 'Flow_Rate_bbl_day', 'Permeability_mD', 'Porosity_pct']
✅ پارامترهای نرمال‌سازی ذخیره شدند.
✅ داده نرمال‌سازی‌شده ذخیره شد.
